In [2]:
#import packages
import argparse
import pandas as pd
import h5py
import numpy as np
import scipy as sp
import scipy.stats as stats
import scipy.io
import nibabel.freesurfer.mghformat as mgh
import scipy.io
import itertools 
import pickle
import sys

utils_dir = '/oak/stanford/groups/kalanit/biac2/kgs/projects/Dawn/NSD/code/streams/utils/'
sys.path.append(utils_dir)

from rsm_utils import get_flat_lower_tri, get_reliability_data 

data_dir = '/oak/stanford/groups/kalanit/biac2/kgs/projects/Dawn/NSD/data/'
local_data_dir = '/oak/stanford/groups/kalanit/biac2/kgs/projects/Dawn/NSD/local_data/'

def fast_pearson(x,y):
    #faster, vectorized version
    xz = x - x.mean(axis=0)
    yz = y - y.mean(axis=0)
    xzss = (xz * xz).sum(axis=0)
    yzss = (yz * yz).sum(axis=0)
    r = np.matmul(xz.transpose(), yz) / (np.sqrt(np.outer(xzss, yzss)) + np.finfo(float).eps) #precision issues
    return np.maximum(np.minimum(r, 1.0), -1.0) #for precision issues

In [1]:
subjid = '05'
hemi = 'rh'
roi_name = 'tessellate_10vox_shrink5'
min_idx = 5
max_idx = 6

In [3]:
chunks = list(range(min_idx,max_idx))

print(chunks)

[5]


In [4]:

print(subjid)

n_repeats = 3
convert_to_int = 10000

#get ROI data
parcels = []
mgh_file = mgh.load(local_data_dir+'freesurfer/subj'+ subjid +'/' + hemi + '.' + roi_name + '.mgz')
parcels.append(mgh_file.get_fdata()[:,0,0])

num_rois = int(np.max(parcels))

#get trial ids and mask        
all_ids = []
max_session = np.zeros(len([subjid]))
for sidx, sid in enumerate([subjid]):

    data = pd.read_csv(data_dir+'nsddata/ppdata/subj'+ sid +'/behav/responses.tsv', sep='\t')

    max_session[sidx] = np.max(np.array(data['SESSION'])) 

    all_ids.append(np.array(data['73KID']))

which_reps = []
for sidx, sid in enumerate([subjid]):
    vals, idx_start, count = np.unique(all_ids[sidx], return_counts=True,
                                    return_index=True)
    which_reps.append(vals[count == n_repeats])

least_trials = min(which_reps, key=len)

id_nums_3reps = []
mask_3reps = []
for sidx, sid in enumerate([subjid]):

    data = pd.read_csv(data_dir+'nsddata/ppdata/subj'+ sid +'/behav/responses.tsv', sep='\t')

    mask_3reps.append(np.isin(all_ids[sidx],which_reps[sidx]))
    id_nums_3reps.append(np.array(data['73KID'])[mask_3reps[sidx]])

arr1inds = id_nums_3reps[sidx].argsort()

#get and sort z-scored betas
betas_by_ROI = [[] for j in range(num_rois)]

05


In [5]:
num_rois

3821

In [6]:
len(parcels[0])

198908

In [7]:
max_session

array([40.])

In [8]:
for sidx, sid in enumerate([subjid]):
    print(sid)

05


In [9]:
for sidx, sid in enumerate([subjid]):
        
    mask = mask_3reps[sidx]
    sorted_betas = []

    #get all betas across all sessions
    for sess in range(1,int(max_session[sidx])+1):
        print(sess)

        if(sess < 10):
            idx = '0' + str(sess)
        else:
            idx = str(sess)

        raw_betas = h5py.File(local_data_dir+'freesurfer/subj'+sid+'/betas/'+ hemi +'.zscore_betas_session'+idx+'.hdf5','r')

        sess_betas = raw_betas['zscore_betas'][:][mask[(sess-1)*750:sess*750]]
        del raw_betas

        if(sess==1):
            for roi_idx in range(num_rois):
                betas_by_ROI[roi_idx] = sess_betas[:,parcels[sidx] == roi_idx+1]
        else:
            for roi_idx in range(num_rois):
                betas_by_ROI[roi_idx] = np.append(betas_by_ROI[roi_idx],sess_betas[:,parcels[sidx] == roi_idx+1],axis=0)

        del sess_betas

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


In [10]:
betas_by_repeat_by_ROI = [[[] for j in range(num_rois)] for i in range(len([subjid]))]
for sidx, sid in enumerate([subjid]):
    for roi_idx in range(num_rois):  

        sorted_betas = betas_by_ROI[roi_idx][arr1inds[::-1]]

        for r in range(n_repeats):
            betas_by_repeat_by_ROI[sidx][roi_idx].insert(r,sorted_betas[r::3])

In [11]:
sorted_betas = betas_by_ROI[11][arr1inds[::-1]]
if sorted_betas.size == 0:
    sorted_betas = np.zeros_like(betas_by_ROI[roi_idx-1][arr1inds[::-1]])

In [ ]:
for roi_idx in range(num_rois):  
    if betas_by_repeat_by_ROI[sidx][roi_idx][0].size == 0:
        print(roi_idx)

In [ ]:
#Create RSMS for all the ROIs, repeats and subjects (moved to within mega matrix loop for mem efficiency)
tril_flat_shape = int((betas_by_repeat_by_ROI[0][0][0].shape[0]**2/2) - (betas_by_repeat_by_ROI[0][0][0].shape[0]/2))
sidx = 0 #currently doing one subject at a time

r1_trial_order = [0, 0, 1, 1, 2, 2]
r2_trial_order = [1, 2, 0, 2, 0, 1]

print('starting mega matrix')
#make the mega matrix!
mega_matrix = np.zeros((len(chunks),num_rois))
m_idx = 0 #allow for chunks not starting at 0

for cidx, roi_idx1 in enumerate(chunks): #rows - i.e. model candidate
        
    flat_rsm_roi1 = np.zeros((tril_flat_shape, n_repeats))
    for r in range(n_repeats):
        flip = betas_by_repeat_by_ROI[sidx][roi_idx1][r].T
        rsm = fast_pearson(flip,flip)
        flat_rsm_roi1[:, r] = get_flat_lower_tri(rsm,diagonal=False)

    split_half = np.zeros((3))
    split_half = [fast_pearson(flat_rsm_roi1[:,0],flat_rsm_roi1[:,1])[0][0],
                fast_pearson(flat_rsm_roi1[:,0],flat_rsm_roi1[:,2])[0][0],
                fast_pearson(flat_rsm_roi1[:,1],flat_rsm_roi1[:,2])[0][0]]
    NC_model = np.abs(np.mean(split_half) * 100) #can't have neg NC

    flat_rsm_roi2 = np.zeros((tril_flat_shape, n_repeats)) #initialize once then rewrite for time
    rsm_corr = np.zeros((6))
    for roi_idx2 in range(num_rois): #columns - i.e. target data

        
        if betas_by_repeat_by_ROI[sidx][roi_idx2][0].size == 0: #no betas
            print(roi_idx2)
            mega_matrix[m_idx,roi_idx2] = 0
        else:
            for r in range(n_repeats):
                flip = betas_by_repeat_by_ROI[sidx][roi_idx2][r].T
                rsm = fast_pearson(flip,flip)
                flat_rsm_roi2[:, r] = get_flat_lower_tri(rsm,diagonal=False)

            split_half = [fast_pearson(flat_rsm_roi2[:,0],flat_rsm_roi2[:,1])[0][0],
                        fast_pearson(flat_rsm_roi2[:,0],flat_rsm_roi2[:,2])[0][0],
                        fast_pearson(flat_rsm_roi2[:,1],flat_rsm_roi2[:,2])[0][0]]
            NC_target = np.abs(np.mean(split_half) * 100) #can't have neg NC

            for r in range(6):
                rsm_corr[r] = fast_pearson(flat_rsm_roi2[:, r1_trial_order[r]],
                                            flat_rsm_roi2[:, r2_trial_order[r]])[0][0]

            mega_matrix[m_idx,roi_idx2] = np.mean(rsm_corr) * np.sqrt(100/NC_model) * np.sqrt(100/NC_target)
    
    m_idx += 1

starting mega matrix
11
149
550
557
573
870
899


/oak/stanford/groups/kalanit/biac2/kgs/projects/Dawn/NSD_venv/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: divide by zero encountered in double_scalars
/oak/stanford/groups/kalanit/biac2/kgs/projects/Dawn/NSD_venv/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
#save to local data folder
save_file = local_data_dir + 'processed/' + subjid + '/' + hemi + '_' + roi_name + '_' + str(min_idx) + 'to' + str(max_idx) + '.data'

with open(save_file, 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump([mega_matrix], filehandle)

In [ ]:
roi_idx2=11


for r in range(n_repeats):
    print(r)
    flip = betas_by_repeat_by_ROI[sidx][roi_idx2][r].T
    rsm = fast_pearson(flip,flip)
    flat_rsm_roi2[:, r] = get_flat_lower_tri(rsm,diagonal=False)

        

In [ ]:
r = 0
flip = betas_by_repeat_by_ROI[sidx][roi_idx2][r].T

In [ ]:
betas_by_repeat_by_ROI[0][11]

In [ ]:
mega_matrix

In [ ]:
print(roi_idx1)
flat_rsm_roi1 = np.zeros((tril_flat_shape, n_repeats))
for r in range(n_repeats):
    flip = betas_by_repeat_by_ROI[sidx][roi_idx1][r].T
    rsm = fast_pearson(flip,flip)
    flat_rsm_roi1[:, r] = get_flat_lower_tri(rsm,diagonal=False)

split_half = np.zeros((3))
split_half = [fast_pearson(flat_rsm_roi1[:,0],flat_rsm_roi1[:,1])[0][0],
            fast_pearson(flat_rsm_roi1[:,0],flat_rsm_roi1[:,2])[0][0],
            fast_pearson(flat_rsm_roi1[:,1],flat_rsm_roi1[:,2])[0][0]]
NC_model = np.mean(split_half) * 100

In [ ]:
NC_model

In [ ]:
np.sqrt(100/NC_model)

In [ ]:
roi_idx2 = 180

In [ ]:
r=0

In [ ]:
betas_by_repeat_by_ROI[sidx][roi_idx2][r].T 

In [ ]:
flip = betas_by_repeat_by_ROI[sidx][roi_idx2][r].T 
rsm = fast_pearson(flip,flip)


In [ ]:
x= flip
y = flip

In [ ]:
#faster, vectorized version
xz = x - x.mean(axis=0)
yz = y - y.mean(axis=0)


In [ ]:

xzss = (xz * xz).sum(axis=0)
yzss = (yz * yz).sum(axis=0)

In [ ]:
np.sqrt(np.min(np.outer(xzss, yzss)))

In [ ]:
r = np.matmul(xz.transpose(), yz) / np.sqrt(np.outer(xzss, yzss))
    

In [ ]:
flip.shape

In [ ]:
flat_rsm_roi2 = np.zeros((tril_flat_shape, n_repeats))
for r in range(n_repeats):
    flip = betas_by_repeat_by_ROI[sidx][roi_idx2][r].T
    rsm = fast_pearson(flip,flip)
    flat_rsm_roi2[:, r] = get_flat_lower_tri(rsm,diagonal=False)


In [ ]:
split_half = np.zeros((3))
split_half = [fast_pearson(flat_rsm_roi2[:,0],flat_rsm_roi2[:,1])[0][0],
            fast_pearson(flat_rsm_roi2[:,0],flat_rsm_roi2[:,2])[0][0],
            fast_pearson(flat_rsm_roi2[:,1],flat_rsm_roi2[:,2])[0][0]]
NC_target = np.abs(np.mean(split_half) * 100)


In [ ]:
np.mean(split_half)

In [ ]:
100/NC_target

In [ ]:
fast_pearson(flat_rsm_roi2[:,0],flat_rsm_roi2[:,1])[0][0]

In [ ]:
stats.pearsonr(flat_rsm_roi2[:,1],flat_rsm_roi2[:,2])[0]

In [ ]:
rsm_corr = np.zeros((6))
for r in range(6):
    rsm_corr[r] = fast_pearson(flat_rsm_roi2[:, r1_trial_order[r]],
                                flat_rsm_roi2[:, r2_trial_order[r]])[0][0]

In [ ]:
rsm_corr

In [ ]:
np.mean(rsm_corr) * np.sqrt(100/NC_model) * np.sqrt(100/NC_target)

In [ ]:
np.sqrt(100/NC_model)

In [ ]:
np.sqrt(100/NC_target)